In [1]:
import ccxt
import time

# API Keys (Replace with actual keys)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to exchanges
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})


In [ ]:
# Settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
LEVERAGE = 5
INVESTMENT_USDT = 1000

TRADING_FEES = {
    'Bybit': 0.001,   # 0.1%
    'Bitget': 0.0008  # 0.08%
}

def get_tradable_spot_tokens(exchange, name):
    try:
        markets = exchange.load_markets()
        tokens = []
        for symbol, market in markets.items():
            if '/USDT' in symbol and ':USDT' not in symbol:
                try:
                    ticker = exchange.fetch_ticker(symbol)
                    if ticker.get('quoteVolume', 0) >= MIN_VOLUME:
                        tokens.append(symbol)
                except:
                    continue
        print(f"✅ {len(tokens)} spot tokens on {name}")
        return tokens
    except Exception as e:
        print(f"❌ Error loading markets from {name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        return ticker.get('last', None)
    except Exception as e:
        print(f"❌ Price error {symbol}: {e}")
        return None

def get_avg_orderbook_price(exchange, symbol, side, usdt_amount):
    try:
        depth = exchange.fetch_order_book(symbol)
        orders = depth['asks'] if side == 'buy' else depth['bids']
        total_cost = 0
        total_qty = 0
        for price, qty in orders:
            trade_cost = price * qty
            if total_cost + trade_cost >= usdt_amount:
                remaining = usdt_amount - total_cost
                total_qty += remaining / price
                total_cost += remaining
                break
            total_cost += trade_cost
            total_qty += qty
        avg_price = total_cost / total_qty if total_qty else 0
        return avg_price, total_qty
    except Exception as e:
        print(f"❌ Order book error for {symbol}: {e}")
        return None, None

def check_inter_exchange_arbitrage(spot_ex, spot_name, futures_ex, futures_name, token):
    spot_price = get_price(spot_ex, token)
    futures_symbol = token.replace('/USDT', '/USDT:USDT')

    if futures_symbol not in futures_ex.load_markets():
        return

    futures_price = get_price(futures_ex, futures_symbol)
    if not spot_price or not futures_price:
        return

    fee_total = TRADING_FEES[spot_name] + TRADING_FEES[futures_name]
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - fee_total

    if net_profit >= SPREAD_THRESHOLD:
        token_qty = (INVESTMENT_USDT * LEVERAGE) / spot_price
        avg_spot_price, spot_qty = get_avg_orderbook_price(spot_ex, token, 'buy', INVESTMENT_USDT)
        avg_futures_price, futures_qty = get_avg_orderbook_price(futures_ex, futures_symbol, 'sell', INVESTMENT_USDT)

        print(f"\n🚀 Inter-Exchange Arbitrage Found ({spot_name} Spot → {futures_name} Futures)")
        print(f"🔹 Token: {token}")
        print(f"🔹 Spot Price: {spot_price:.4f} USDT ({spot_name})")
        print(f"🔹 Futures Price: {futures_price:.4f} USDT ({futures_name})")
        print(f"🔹 Spread: {spread:.4%}, Net Profit: {net_profit:.4%}")
        if avg_spot_price and avg_futures_price:
            print(f"📊 Orderbook Spot Buy: {avg_spot_price:.4f} for {spot_qty:.4f}")
            print(f"📊 Orderbook Futures Sell: {avg_futures_price:.4f} for {futures_qty:.4f}")
        print(f"💰 Est. Profit on ${INVESTMENT_USDT}: ${(net_profit * INVESTMENT_USDT):.2f}")
        print("⚡ Ready to execute arbitrage\n")

def main():
    while True:
        try:
            bybit_tokens = get_tradable_spot_tokens(bybit, "Bybit")
            bitget_tokens = get_tradable_spot_tokens(bitget, "Bitget")

            shared_tokens = set(bybit_tokens).intersection(set(bitget_tokens))
            if not shared_tokens:
                print("❌ No shared tokens to check. Waiting...")
                time.sleep(10)
                continue

            print(f"\n🔍 Checking arbitrage on {len(shared_tokens)} tokens...")
            for token in shared_tokens:
                # Bybit Spot -> Bitget Futures
                check_inter_exchange_arbitrage(bybit, "Bybit", bitget, "Bitget", token)
                # Bitget Spot -> Bybit Futures
                check_inter_exchange_arbitrage(bitget, "Bitget", bybit, "Bybit", token)

            print("✅ Cycle complete. Waiting for next check...\n")
            time.sleep(CHECK_INTERVAL)

        except Exception as e:
            print(f"❌ Main loop error: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

✅ 432 spot tokens on Bybit
✅ 554 spot tokens on Bitget

🔍 Checking arbitrage on 268 tokens...
✅ Cycle complete. Waiting for next check...

✅ 432 spot tokens on Bybit
✅ 554 spot tokens on Bitget

🔍 Checking arbitrage on 268 tokens...
✅ Cycle complete. Waiting for next check...

✅ 430 spot tokens on Bybit
✅ 554 spot tokens on Bitget

🔍 Checking arbitrage on 268 tokens...
✅ Cycle complete. Waiting for next check...

✅ 430 spot tokens on Bybit
✅ 552 spot tokens on Bitget

🔍 Checking arbitrage on 266 tokens...

🚀 Inter-Exchange Arbitrage Found (Bitget Spot → Bybit Futures)
🔹 Token: PIRATE/USDT
🔹 Spot Price: 0.1056 USDT (Bitget)
🔹 Futures Price: 0.1064 USDT (Bybit)
🔹 Spread: 0.7102%, Net Profit: 0.5302%
📊 Orderbook Spot Buy: 0.1058 for 9449.3251
📊 Orderbook Futures Sell: 0.1061 for 9425.4230
💰 Est. Profit on $1000: $5.30
⚡ Ready to execute arbitrage

✅ Cycle complete. Waiting for next check...

✅ 430 spot tokens on Bybit
✅ 552 spot tokens on Bitget

🔍 Checking arbitrage on 266 tokens...
❌ Pr